<H1>Ramba Test</h1>

Following cell loads sets up environment variables, loads ipyparallel, and starts a set of MPI processes.  It shows examples of common options, which are specified as environment variables.  Uncomment and update these as needed if you wish to override defaults.  Note: These environment variables must be set before the first time that ramba is imported.  To rerun with different settings, remember to restart the IPython Kernel.  

In [ ]:
N = 2   # number of mpi processes;  should be set to 2 or more (otherwise ramba will use ray)

#Number of threads per process (defaults to 1 when using MPI)
%set_env RAMBA_NUM_THREADS=4

#set to 1 to forward calls of reshape to reshape_copy
#%set_env RAMBA_RESHAPE_COPY=1

#set to 1 if you need any dimension to exceed 2 billion (64-bit vs 32-bit indices)
#%set_env RAMBA_BIG_DATA=1

#set to 0,1,2,3,.. to set debugging log level
#%set_env RAMBA_DEBUG=1

#set to 1 to get detailed timing information
#%set_env RAMBA_TIMING=1

#set to worker number i -- this worker will print detailed timing if RAMBA_TIMING=1
#%set_env RAMBA_TIMING_WORKER=i


import ipyparallel as ipp
c = await ipp.Cluster(engines="mpi").start_and_connect(n=N, activate=True)

%autopx   # tell ipyparallel to start treating subsequent cells as code that will run in all of the mpi processes

In [ ]:
print ("hi")  # sanity check -- should produce N outputs, one from each mpi process

Next, load ramba

In [ ]:
import ramba as np
import time as time
np.sync()

In [ ]:
A = np.arange(100*1000*1000)/1000.0
np.sync()

In [ ]:
for i in range(5):
    t0 = time.time()
    B = np.sin(A)
    C = np.cos(A)
    D = B*B + C**2
    np.sync()
    t1 = time.time()
    if np.is_main_thread:   # check if rank 0 so only one process prints
        print ("Iteration",i+1,"time:",t1-t0)

Now let's try again with numpy

In [ ]:
%autopx   # toggle running in all processes

In [ ]:
%%time
import numpy as np2
import time as time
A2 = np2.arange(100*1000*1000)/1000.0

In [ ]:
for i in range(5):
    t0 = time.time()
    B2 = np2.sin(A2)
    C2 = np2.cos(A2)
    D2 = B2*B2 + C2**2
    t1 = time.time()
    print ("Iteration",i+1,"time:",t1-t0)